# Instructions  
You will submit an HTML document to Canvas as your final version.  

Your document should show your code chunks/cells as well as any output. Make sure that only relevant output is printed. Do not, for example, print the entire dataset in your final rendered file.  

Your document should also be clearly organized, so that it is easy for a reader to find your answers to each question.  

# The Data  
This week, we consider a dataset generated from text data.  

The original dataset can be found here: https://www.kaggle.com/datasets/kingburrito666/cannabis-strains. It consists of user reviews of different strains of cannabis. Users rated their experience with the cannabis strain on a scale of 1 to 5. They also selected words from a long list to describe the Effects and the Flavor of the cannabis.  

In the dataset linked above, each row is one strain of cannabis. The average rating of all testers is reported, as well as the most commonly used words for the effect and flavor.  

Some data cleaning has been performed for you: The Effect and Flavor columns have been converted to dummy variables indicating if the particular word was used for the particular strain.  

This cleaned data can be found at: https://www.dropbox.com/s/s2a1uoiegitupjc/cannabis_full.csv  

Our goal will be to fit models that identify the Sativa types from the Indica types, and then to fit models that also distinguish the Hybrid types.  

IMPORTANT: In this assignment, you do not need to consider different feature sets. Normally, this would be a good thing to try - but for this homework, simply include all the predictors for every model.  

# Part One: Binary Classification  
Create a dataset that is limited only to the Sativa and Indica type cannabis strains.  

This section asks you to create a final best model for each of the four new model types studied this week: LDA, QDA, SVC, and SVM. For SVM, you may limit yourself to only the polynomial kernel.  

For each, you should:  
- Choose a metric you will use to select your model, and briefly justify your choice. (Hint: There is no specific target category here, so this should not be a metric that only prioritizes one category.)  

- Find the best model for predicting the Type variable. Don’t forget to tune any hyperparameters.  

- Report the (cross-validated!) metric.  

- Fit the final model.  

- Output a confusion matrix.  

## Q1: LDA

## Q2: QDA

## Q3: SVC

## Q4: SVM

# Part Two: Natural Multiclass  
Now use the full dataset, including the Hybrid strains.  

## Q1  
Fit a decision tree, plot the final fit, and interpret the results.

## Q2  
Repeat the analyses from Part One for LDA, QDA, and KNN.  

## Q3  
Were your metrics better or worse than in Part One? Why? Which categories were most likely to get mixed up, according to the confusion matrices? Why?

# Part Three: Multiclass from Binary  
Consider two models designed for binary classification: SVC and Logistic Regression.

## Q1  
Fit and report metrics for OvR versions of the models. That is, for each of the two model types, create three models:  

- Indica vs. Not Indica  

- Sativa vs. Not Sativa  

- Hybrid vs. Not Hybrid  

## Q2  
Which of the six models did the best job distinguishing the target category from the rest? Which did the worst? Does this make intuitive sense?

## Q3  
Fit and report metrics for OvO versions of the models. That is, for each of the two model types, create three models:  

- Indica vs. Sativa  

- Indica vs. Hybrid  

- Hybrid vs. Sativa  

## Q4  
Which of the six models did the best job distinguishing at differentiating the two groups? Which did the worst? Does this make intuitive sense?

## Q5  
Suppose you had simply input the full data, with three classes, into the LogisticRegression function. Would this have automatically taken an “OvO” approach or an “OvR” approach?  

What about for SVC?  

Note: You do not actually have to run code here - you only need to look at sklearn’s documentation to see how these functions handle multiclass input.  